In [1]:
import ee
import geopandas as gpd
import numpy as np
import geemap

In [2]:
#Authentication
ee.Authenticate()

True

In [3]:
#Initialise
ee.Initialize()

In [4]:
#create map
m = geemap.Map()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [5]:
# Load the Counties feature collection
asset_path = 'projects/ee-heat-001/assets/kenyan-counties'
kenya_counties = ee.FeatureCollection(asset_path)

In [6]:
#Filter for Mombasa county
mombasa = kenya_counties.filter(ee.Filter.eq('COUNTY', 'Mombasa'))

#Add it as a layer on the map
m.addLayer(mombasa, {}, 'Mombasa County')

In [7]:
# Load MODIS/MOD11A2
collection = ee.ImageCollection('MODIS/061/MOD11A2') \
    .filterDate('2024-01-01', '2024-10-01') \
    .filterBounds(mombasa)

# Function to transform T in Kelvin using scaling factor as provided with the link
def convertToC(image):
    result = image.multiply(0.02).subtract(273.15)
    result = result.copyProperties(image, ['system:time_start'])
    return result

collectionCelcius = collection.map(convertToC)

# Calculate mean LST in the Day
LSTmean = collectionCelcius.select('LST_Day_1km').mean()
m.addLayer(LSTmean.clip(mombasa), {
    "min": 20, "max": 40,
    "palette": ['green', 'yellow', 'darkorange', 'red']
}, 'Mean temperature Day')


In [8]:
# Calculate mean LST in the Night
LSTmean = collectionCelcius.select('LST_Night_1km').mean()

#Add it as a layer on the map
m.addLayer(LSTmean.clip(mombasa), {
    "min": 20, "max": 40,
    "palette": ['green', 'yellow', 'darkorange', 'red']
}, 'Mean temperature Night')

In [10]:
# import json

# # Get the collection info
# collection_info = collection.getInfo()

# # Pretty-print the JSON data
# print(json.dumps(collection_info, indent=2))

In [9]:
#Create another map for land surface temperature
m2 = geemap.Map()
m2

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [10]:
#Add Mombasa as a layer on the map
m2.addLayer(mombasa, {}, 'Mombasa County')

In [14]:
# Function to Mask Clouds and Cloud Shadows in Landsat 8 Imagery

def cloudMask(image):
    # Define cloud shadow and cloud bitmasks (Bits 3 and 4)
    cloudShadowBitmask = (1 << 3)
    cloudBitmask = (1 << 4)

    # Select the Quality Assessment (QA) band for pixel quality information
    qa = image.select('QA_PIXEL')

    # Create a binary mask to identify clear conditions (both cloud and cloud shadow bits set to 0)
    mask = qa.bitwiseAnd(cloudShadowBitmask).eq(0).And(qa.bitwiseAnd(cloudBitmask).eq(0))

    # Update the original image, masking out cloud and cloud shadow-affected pixels
    return image.updateMask(mask)


In [15]:
# Import landsat imagery

dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(mombasa).filterDate('2024-01-01', '2024-10-01').map(cloudMask)

# Applies scaling factors.
def applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True) \
    .addBands(thermalBands, None, True)
dataset = dataset.map(applyScaleFactors).median()

visualization = {
    "bands": ['SR_B4', 'SR_B3', 'SR_B2'],
    "min": 0.0,
    "max": 0.3,
}

m2.addLayer(dataset.clip(mombasa), visualization, 'True Color ')

In [16]:
# Calculate Normalized Difference Vegetation Index (NDVI)
ndvi = dataset.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

# Define NDVI Visualization Parameters
ndvi_palette = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}

# Add NDVI layer to the map
m2.addLayer(ndvi.clip(mombasa), ndvi_palette, 'NDVI Mombasa')

In [17]:
# Calculate the minimum NDVI value within Mombasa
ndvi_min = ndvi.reduceRegions(
    reducer=ee.Reducer.min(),
    collection=mombasa,
    scale=30,
).first().get('min').getInfo()
# Calculate the maximum NDVI value within Mombasa
ndvi_max = ndvi.reduceRegions(
    reducer=ee.Reducer.max(),
    collection=mombasa,
    scale=30,
).first().get('max').getInfo()

print(f"Minimum NDVI: {ndvi_min}")
print(f"Maximum NDVI: {ndvi_max}")

Minimum NDVI: -0.8521126760563386
Maximum NDVI: 0.9428916191311979


In [27]:
# Fraction of Vegetation (FV) Calculation
# Formula: ((NDVI - NDVI_min) / (NDVI_max - NDVI_min))^2
ndvi_min_ee = ee.Number(ndvi_min)
ndvi_max_ee = ee.Number(ndvi_max)

fv = ndvi.subtract(ndvi_min_ee) \
         .divide(ndvi_max_ee.subtract(ndvi_min_ee)) \
         .pow(2) \
         .rename('FV')

# Emissivity Calculation
# Formula: 0.004 * FV + 0.986
em = fv.multiply(0.004).add(0.986).rename('EM')

# To view or export the results:
# Print the FV and EM images to check
print("Fraction of Vegetation (FV):", fv.getInfo())
print("Emissivity (EM):", em.getInfo())

Fraction of Vegetation (FV): {'type': 'Image', 'bands': [{'id': 'FV', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
Emissivity (EM): {'type': 'Image', 'bands': [{'id': 'EM', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [28]:
# Select Thermal Band (Band 10) and Rename It
thermal = dataset.select('ST_B10').rename('thermal')

In [51]:
# Calculate the land surface temperature (LST)
# Formula: (TB / (1 + (λ * (TB / 1.438)) * ln(em))) - 273.15
lst = thermal.expression(
    '(TB / (1 + (0.00115 * (TB / 1.438)) * log(em))) - 273.15', {
        'TB': thermal.select('thermal'),  # Select the thermal band (TB)
        'em': em  # Assign emissivity (em)
    }).rename('LST Mombasa')

# Add the newly calculated LST band to the original image
dataset_with_lst = dataset.addBands(lst)

# Add the LST Layer to the Map with Custom Visualization Parameters
vis_params = [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
]

m2.addLayer(lst.clip(mombasa), {
    'min': 27,  # Minimum LST value
    'max': 47,  # Maximum LST value
    'palette': vis_params
}, 'Land Surface Temperature')

In [30]:
m2

Map(bottom=268249.0, center=[-3.982765858235823, 39.70149993896485], controls=(WidgetControl(options=['positio…

In [45]:
# Get the band names of the dataset
band_names = dataset_with_lst.bandNames().getInfo()
print("Available bands:", band_names)

Available bands: ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10', 'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT', 'LST Mombasa']


In [37]:
#Check crs of the image
projection = dataset_with_lst.projection().getInfo()
projection

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}

In [55]:
# Export the image locally
vis_params_dict = {
    'min': 27,  # Minimum LST value
    'max': 47,  # Maximum LST value
    'palette': [
        '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
        '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
        '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
        'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
        'ff0000', 'de0101', 'c21301', 'a71001', '911003'
    ]
}

geemap.ee_export_image(
    dataset_with_lst.select('LST Mombasa').clip(mombasa).visualize(**vis_params_dict),
    filename="Msa_LST_Viz.tif",
    scale=30,  # Set the scale appropriately
    region=mombasa.geometry(),
    crs='EPSG:4326'  # Set the correct CRS
)

Generating URL ...
Please wait ...
Data downloaded to /workspaces/Mombasa-UHI/Msa_LST_Viz.tif


In [48]:
# Calculate the minimum LST value within Mombasa
min_lst = dataset_with_lst.select('LST Mombasa').reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=mombasa.geometry(),
    scale=30
).get('LST Mombasa').getInfo()

# Calculate the maximum LST value within Mombasa
max_lst = dataset_with_lst.select('LST Mombasa').reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=mombasa.geometry(),
    scale=30
).get('LST Mombasa').getInfo()

print("Minimum LST Mombasa:", min_lst)
print("Maximum LST Mombasa:", max_lst)


Minimum LST Mombasa: 27.015069548658232
Maximum LST Mombasa: 47.94628006708251


In [35]:
#Get the needed components for export
crs = projection["crs"]
crs_transform = projection["transform"]

In [36]:
# Uncomment this if you prefer using the bounding box
# Define the bounding box for Mombasa
# bounding_box = ee.Geometry.Rectangle([39.5629970643, -4.15227602331, 39.762889862, -3.92034167672])

# Select the thermal band and clip it to Mombasa
final_image = dataset_with_lst.select('LST Mombasa').clip(mombasa)

In [37]:
#Export the image
geemap.ee_export_image(
    final_image,
    filename="Mombasa_Land_Surface_Temperature_2024.tif",
    crs=crs,
    crs_transform=crs_transform,
    region=mombasa.geometry(),
)

Generating URL ...
Please wait ...
Data downloaded to /workspaces/Mombasa-UHI/Mombasa_Land_Surface_Temperature_2024.tif


In [38]:
#Export the image to Google Drive
geemap.ee_export_image_to_drive(
    final_image, description="Mombasa_Land_Surface_Temperature_2024", folder="export", region=mombasa.geometry(), scale=30
)

In [21]:
#read the tif using rasterio for inpection
import rasterio as rio

# Read the raster file
with rio.open('Mombasa_Land_Surface_Temperature_2024.tif') as src:
    # Read the raster data
    raster_data = src.read(1)
    
    # Read additional information
    profile = src.profile
    bounds = src.bounds
    resolution = src.res
    mean_val = raster_data.mean()
    min_val = raster_data.min()
    max_val = raster_data.max()

# Print the extracted information
print("Profile:", profile)
print("Bounds:", bounds)
print("Resolution:", resolution)
print("Mean value:", mean_val)
print("Minimum value:", min_val)
print("Maximum value:", max_val)

Profile: {'driver': 'GTiff', 'dtype': 'float64', 'nodata': 0.0, 'width': 1, 'height': 2, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(1.0, 0.0, 39.0,
       0.0, 1.0, -5.0), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'deflate', 'interleave': 'band'}
Bounds: BoundingBox(left=39.0, bottom=-3.0, right=40.0, top=-5.0)
Resolution: (1.0, 1.0)
Mean value: 33.705019981322835
Minimum value: 27.268528001244988
Maximum value: 40.14151196140068
